In [47]:
import pandas as pd
import string

## Читаем текстовые описания из файла, очищаем данные для дальнейшей обработки и сохраняем в файл

Набор данных Flickr30k стал стандартным эталоном для описания изображений на основе предложений.
Набор скачан с https://www.kaggle.com/datasets/hsankesara/flickr-image-dataset?resource=download

In [48]:
#Основной файл results.csv он содержит название изображения, индекс описания
# и описания для этого изображения. Каждое изображения содержит по 5 описаний.
# Между собой поля отделены символом |

dataset_path = "D:/Temp/Dataset/kaggle/flickr_30k" # основной путь к датасету
captions_path = dataset_path + "/results.csv" # путь к файлу с описаниями изображений

df = pd.read_csv(captions_path, delimiter='|') # прочитаем датасет
df.head(10) # выведем первые 10 строк из датасета

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .
5,10002456.jpg,0,Several men in hard hats are operating a gian...
6,10002456.jpg,1,Workers look down from up above on a piece of...
7,10002456.jpg,2,Two men working on a machine wearing hard hats .
8,10002456.jpg,3,Four men on top of a tall structure .
9,10002456.jpg,4,Three men on a large rig .


In [49]:
# Для работы с текстовым описанием изображений необходимо очистить данные от шума
# Для того, чтобы уменьшить размер словесного запаса с которым придется работать
# - все слова в нижний регистр
# - удаляем все знаки препинания
# - удаляем все лова длиной один символ или меньше
# - удаляем все слова с цифрами в них
def clean_text(data):
    idx_to_delete = [] # массив в котором будем хранить индексы для битых строк, которые
                       # надо удалить
    table = str.maketrans('', '', string.punctuation)
    for idx in data.index:
        if pd.notna(data.iat[idx, 2]): # если существует описание изображения
            # разбиваем описание на слова
            desc = data.iat[idx, 2].split()
            # - все лова в нижний регистр
            desc = [word.lower() for word in desc]
            # - удаляем все знаки препинания
            desc = [word.translate(table) for word in desc]
            # - удаляем все слова длиной один символ или меньше
            desc = [word for word in desc if (len(word) > 1)]
            # - удаляем все слова с цифрами в них
            desc = [word for word in desc if (word.isalpha())]
            # преобразуем обратно в строку
            data.iat[idx, 2] = ' '.join(desc)
        else: # если не существует описания изображения, добавляем индекс в массив для удаления
            idx_to_delete.append(idx)
    return idx_to_delete

### Запускаем очистку текста

In [50]:
idxs_to_delete = clean_text(df) # сохраним индексы с битыми описаниями в переменную
df.head(10) # выведем первые 10 строк из датасета

,image_name,comment_number,comment
0,1000092795.jpg,0,two young guys with shaggy hair look at their ...
1,1000092795.jpg,1,two young white males are outside near many bu...
2,1000092795.jpg,2,two men in green shirts are standing in yard
3,1000092795.jpg,3,man in blue shirt standing in garden
4,1000092795.jpg,4,two friends enjoy time spent together
5,10002456.jpg,0,several men in hard hats are operating giant p...
6,10002456.jpg,1,workers look down from up above on piece of eq...
7,10002456.jpg,2,two men working on machine wearing hard hats
8,10002456.jpg,3,four men on top of tall structure
9,10002456.jpg,4,three men on large rig


### Удалим битые описание изображений
Например, в данном датасете были места, где не хватало разделителя | и описание изображения было
пустым. Будем считать что для таких изображение недостаточно описаний для обучения, а поэтому это
изображение надо исключить из рассмотрения

In [55]:
# проверим какова длина массива с битыми строками
print(len(idxs_to_delete))

# возьмем все строки для удаления
df_to_delete = df.loc[idxs_to_delete, :]
# возьмем уникальное имя изображения, т.к. будем удалять все описания для этих изображений
images_to_delete = df_to_delete["image_name"].unique()
# распечатаем имена изображений для удаления
print(images_to_delete)

# удалим все строки с этим изображением
df = df[~df.image_name.isin(images_to_delete)]

# Сохраним датафрейм в файл
path_prepared_captions = dataset_path + "/clean_results_en.csv"
df.to_csv(path_prepared_captions, sep='|', encoding='utf-8', index=False)

1
['2199200615.jpg']
